In [14]:
import pandas as pd 
import numpy as np

#load data into pandas dataframes
data_dir = "../../data/synthea/"
conditions_file = data_dir+"conditions.csv"
medications_file = data_dir+"medications.csv"
observatios_file = data_dir+"observations.csv"
patients_file = data_dir+"patients.csv"

df_cond = pd.read_csv(conditions_file)
df_med = pd.read_csv(medications_file)
df_obs = pd.read_csv(observatios_file)
df_pat = pd.read_csv(patients_file)

In [15]:
df_cond.head()

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
0,2012-03-31,2012-04-30,7d3e489a-7789-9cd6-2a1b-711074af481b,1f2b8067-61bd-88ca-a497-b177756efe62,307731004,Injury of tendon of the rotator cuff of shoulder
1,2014-10-08,2014-10-17,7d3e489a-7789-9cd6-2a1b-711074af481b,c0043d0a-e6b1-7d0a-ab72-263d9591b1b1,195662009,Acute viral pharyngitis (disorder)
2,2017-12-08,2017-12-15,7d3e489a-7789-9cd6-2a1b-711074af481b,9a2ce31d-bf2d-0f0e-f5e9-945602e19b0c,444814009,Viral sinusitis (disorder)
3,2020-03-15,2020-03-29,7d3e489a-7789-9cd6-2a1b-711074af481b,1402ddca-c6d3-3bf0-2369-997840511cfb,49727002,Cough (finding)
4,2020-03-15,2020-03-29,7d3e489a-7789-9cd6-2a1b-711074af481b,1402ddca-c6d3-3bf0-2369-997840511cfb,248595008,Sputum finding (finding)


In [16]:
df_med.head()

,START,STOP,PATIENT,PAYER,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,PAYER_COVERAGE,DISPENSES,TOTALCOST,REASONCODE,REASONDESCRIPTION
0,1989-12-09T22:06:58Z,NaN,a3795ec8-54f3-e99e-a4b1-4c067f3141d7,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,959d732e-e052-d01d-4b6d-428a208c93fd,106258,Hydrocortisone 10 MG/ML Topical Cream,5.05,0.0,375,1893.75,40275004.0,Contact dermatitis
1,1989-12-23T22:35:58Z,NaN,a3795ec8-54f3-e99e-a4b1-4c067f3141d7,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,7c1f93ba-a68f-03b3-3999-38ea56e424c2,141918,Terfenadine 60 MG Oral Tablet,7.97,0.0,375,2988.75,NaN,NaN
2,1989-12-23T22:35:58Z,NaN,a3795ec8-54f3-e99e-a4b1-4c067f3141d7,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,7c1f93ba-a68f-03b3-3999-38ea56e424c2,1870230,NDA020800 0.3 ML Epinephrine 1 MG/ML Auto-Inje...,406.87,0.0,375,152576.25,NaN,NaN
3,2015-04-01T22:27:58Z,2015-04-15T22:27:58Z,a3795ec8-54f3-e99e-a4b1-4c067f3141d7,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,3af8003b-e9c7-27aa-cbe9-038d73a3ac21,313782,Acetaminophen 325 MG Oral Tablet,10.03,0.0,1,10.03,10509002.0,Acute bronchitis (disorder)
4,2008-08-27T19:55:43Z,2009-09-02T19:55:43Z,d7acfddb-f4c2-69f4-2081-ad1fb8490448,42c4fca7-f8a9-3cd1-982a-dd9751bf3e2a,2c054c1f-a06a-06a4-c3d3-e33daeaf6560,310798,Hydrochlorothiazide 25 MG Oral Tablet,0.01,0.0,12,0.12,59621000.0,Hypertension


In [17]:
df_obs.head()

,DATE,PATIENT,ENCOUNTER,CODE,DESCRIPTION,VALUE,UNITS,TYPE
0,2011-03-31T02:00:17Z,7d3e489a-7789-9cd6-2a1b-711074af481b,814174f3-2e0e-1625-de48-9c40732c9149,8302-2,Body Height,167.0,cm,numeric
1,2011-03-31T02:00:17Z,7d3e489a-7789-9cd6-2a1b-711074af481b,814174f3-2e0e-1625-de48-9c40732c9149,72514-3,Pain severity - 0-10 verbal numeric rating [Sc...,3.0,{score},numeric
2,2011-03-31T02:00:17Z,7d3e489a-7789-9cd6-2a1b-711074af481b,814174f3-2e0e-1625-de48-9c40732c9149,29463-7,Body Weight,71.1,kg,numeric
3,2011-03-31T02:00:17Z,7d3e489a-7789-9cd6-2a1b-711074af481b,814174f3-2e0e-1625-de48-9c40732c9149,39156-5,Body Mass Index,25.5,kg/m2,numeric
4,2011-03-31T02:00:17Z,7d3e489a-7789-9cd6-2a1b-711074af481b,814174f3-2e0e-1625-de48-9c40732c9149,59576-9,Body mass index (BMI) [Percentile] Per age and...,83.6,%,numeric


In [18]:
df_pat.head()

,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,...,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE
0,7d3e489a-7789-9cd6-2a1b-711074af481b,1993-01-28,NaN,999-95-8631,S99916705,X24646789X,Mr.,Jon665,Pacocha935,NaN,...,Lawrence Massachusetts US,942 Fahey Overpass Apt 21,Natick,Massachusetts,Middlesex County,NaN,42.309347,-71.349633,569019.69,2293.12
1,a3795ec8-54f3-e99e-a4b1-4c067f3141d7,1971-12-01,NaN,999-62-4431,S99941017,X38787090X,Mr.,Dick869,Streich926,NaN,...,Swansea Massachusetts US,1064 Hickle View Apt 7,Chicopee,Massachusetts,Hampden County,1020.0,42.198239,-72.554752,18755.46,0.00
2,3829c803-1f4c-74ed-0d8f-36e502cadd0f,2005-01-07,NaN,999-21-2332,NaN,NaN,NaN,Cordell41,Eichmann909,NaN,...,Chelmsford Massachusetts US,560 Ritchie Way Suite 68,Swansea,Massachusetts,Bristol County,NaN,41.748125,-71.182914,361770.00,2768.96
3,d7acfddb-f4c2-69f4-2081-ad1fb8490448,1990-07-04,NaN,999-53-1990,S99932677,X67053099X,Mrs.,Cheri871,Oberbrunner298,NaN,...,Cambridge Massachusetts US,268 Hansen Loaf Apt 62,Lowell,Massachusetts,Middlesex County,1850.0,42.662520,-71.368933,703332.77,5551.19
4,474766f3-ee93-f5d6-84c3-db38ba803394,2012-04-03,NaN,999-57-2653,NaN,NaN,NaN,Desmond566,O'Conner199,NaN,...,Cohasset Massachusetts US,831 Schumm Lock Apt 62,Westborough,Massachusetts,Worcester County,NaN,42.253951,-71.563825,206450.27,2284.86
